In [1]:
import pandas as pd
import findspark
findspark.init()
from pyspark.sql.functions import regexp_replace
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import mean
from pyspark.sql.functions import col, when

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Project-4").getOrCreate()

In [4]:
# Loading CSV from local to Spark DataFrame
df= spark.read.options(inferSchema="True",delimiter=",",header=True).csv("healthcare-dataset-stroke-data.csv")

df.show(truncate=False)

AnalysisException: Path does not exist: file:/C:/Users/Tanner/Desktop/proj4-team6/healthcare-dataset-stroke-data.csv

In [ ]:
df.count()

In [ ]:
df.columns

In [ ]:
df.printSchema()

In [ ]:
df.select(mean("bmi")).show()

In [ ]:
df = df.withColumn("bmi", col("bmi").cast(DoubleType()))
df2 = df.na.fill(value=28.89, subset=["bmi"])
df2.show()

In [ ]:
df.approxQuantile("bmi", [0.5], 0.0)[0]

In [ ]:
df2.columns

In [ ]:
df2= df2.filter(df.gender!="Other")

In [ ]:
df2 = df2.drop("id")
df2.show()

In [ ]:
df2 = df2.withColumn('work_type', regexp_replace('work_type', 'Never_worked', 'children'))

In [ ]:
df2.select('work_type').distinct().collect()

In [ ]:
condition = col("work_type") == "children"
new_value = "Never"
df2 = df2.withColumn("work_type",when(condition, new_value).otherwise(col("work_type")))

In [ ]:
df2.select("work_type").distinct().show()

In [ ]:
df2.show()

In [ ]:
#csv_file_path = "data.csv"
#df2.write.option("header", True).option("delimiter",",").csv(csv_file_path)